# Predicting Credit Risk for Lending Club

We will explore the full data science life cycle, from data cleaning and feature selection to machine learning. We will focus on credit modelling, that focuses on modeling a borrower's credit risk. 

We will work with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return.

Each borrower fills out an application, providing their past financial history, the reason for the loan etc. Lending Club evaluates each borrower's credit score using past historical data (and their own data science process) and assign an interest rate to the borrower. A higher interest rate means the borrower is riskier, and more unlikely to pay back the loan, while a lower interest rate means the borrower has a good credit history and more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99% and each borrower is given a grade according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

Approved loans are listed on the website, where investors can browse recently approved loans, the borrower's credit score, the purpose for the loan etc. Once they're ready to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the origination fee that Lending Club charges.

The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors. Many loans aren't completely paid off on time, however, and some borrowers default on the loan.

In this project, we will focus on the mindset of a conservative investor, who only wants to invest in loans that have a good chance of being paid off on time. To do that, we need to first understand the features in the dataset, and then experiment with building machine learning models that reliably predict if a loan will be paid off or not.

## Introduction to the Dataset

The approved loans datasets contain information on current loans, completed loans, and defaulted loans. We will define the problem statement for this machine learning project:

- can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

We first need to define what features we want to use, and which column repesents the target column we want to predict. We will read in the data and explore it.

We will focus on approved loans data from 2007 to 2011, since a good number of the loans have already finished. In the datasets for later years, many of the loans are current, and still being paid off.

In [1]:
import pandas as pd
loans_2007 = pd.read_csv(r"C:\projectdatasets\loans_2007.csv")

C:\Users\Chris\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# show the first record
print(loans_2007.iloc[0])

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                            Dec-11
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

In [3]:
# show the number of columns
print(loans_2007.shape[1])

52


In [4]:
# show the number of rows
print(loans_2007.shape[0])

42538


## Removing Unnecessary Columns

There are many columns that are cumbersometo explore all at once. Pay attention to any features that:

- leak information from the future (after the loan has already been funded)
- don't affect a borrower's ability to pay back a loan (e.g. a randomly -generated ID value)
- formatted poorly and requires cleaning
- require more data, or more processing to turn into a useful feature
- contain redundant information

We need to pay attention to data leakage, since it can cause our model to overfit. This is because the model would be using data about the target column that wouldn't be available when using the model on future loans. We need to select one of the columns as the target column to use, for the machine learning phase. We can conclude the following features need to be removed:

- id: randomly generated field for unique identification purposes only
- member_id: randomly generated field for unique identification purposes only
- funded_amnt: leaks data from the future (after loan starts to be funded)
- funded_amnt_inv: same as above
- grade: contains redundant information as the interest rate column (int_rate)
- sub_grade: same above
- emp_title: requires other data and lots of processing to be useful
- issue_d: leaks data from the future (after loan is completed funded)

Lending Club assigns a grade and a sub-grade based on the borrower's interest rate. While the grade and sub_grade values are categorical, the 'int_rate' column contains continuous values, which are better suited for machine learning.

We will drop these columns.

In [5]:
loans_2007 = loans_2007.drop(["id", "member_id", "funded_amnt", 
                              "funded_amnt_inv", "grade", "sub_grade", 
                              "emp_title", "issue_d"], axis=1)

We will also drop the following columns:

- zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible
- out_prncp: leaks data from the future (after loan is being paid off)
- out_prncp_inv: same as above
- total_pymnt: same as above
- total_pymnt_inv: same as above
- total_rec_prncp: same as above

The 'out_prncp' and 'out_prncp_inv' both describe the outstanding principal amount for a loan. These 2 columns, as well as the 'total_pymnt' column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded,  so we don't want to include it.

In [6]:
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", 
                              "total_pymnt", "total_pymnt_inv", 
                              "total_rec_prncp"], axis=1)

We also need to drop the following columns:

- total_rec_int: leaks data from the future (after loan is being paid off)
- total_rec_late_fee: same as above
- recoveries: same as above
- collection_recovery_fee: same as above
- last_pymnt_d: same as above
- last_pymnt_amnt: same as above

All these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower. This information isn't available to an investor before the loan is fully funded, so we don't want to include it.

In [7]:
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", 
                              "collection_recovery_fee", 
                              "last_pymnt_d", "last_pymnt_amnt"], axis=1)

In [8]:
# display the first row
print(loans_2007.iloc[0])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                   Jan-85
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                         83.70%
total_acc                               9
initial_list_status                     f
last_credit_pull_d                

In [9]:
# show the number of columns (now a reduced number)
print(loans_2007.shape[1])

32


## Selecting Target Column

We have reduced the columns from 52 to 32. We now need to decide on a target column to use for modeling.

We should use the 'loan_status' column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values, and we need to convert it to a numerical one for training a model. 

We will explore the different values in this column, and come up with a strategy for converting the values in this column.

In [10]:
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


Explanation for each column:

- Fully Paid - loan fully paid off
- Charged Off - loan for which there is no longer a reasonable expectation of further payments
- Does not meet the credit policy. Status:Fully Paid - while the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved
- Does not meet the credit policy. Status:Charged Off - while the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved
- In Grace Period - loan is past due but still in the grace period of 15 days
- Late (16-30 days) - loan hasn't been paid in 16 to 30 days (late on the current payment)
- Late (31-120 days) - loan hasn't been paid in 31 to 120 days (late on the current payment)
- Current - loan is up to date on current payments
- Default - loan is defaulted, and no payment has been made for more than 121 days

From the investor's perspective, we want to predict which loans will be paid off on time, and which ones won't be. Only the 'Fully Paid' and 'Charged Off' values describe the final outcome of the loan. While the 'Default' status resembles Charged Off, they have essentially no chance of being repaid, while default ones have a small chance.

Since we're interested in predicting which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. We will remove all loans that don't contain either 'Fully Paid' and 'Charged Off' as the loan's status, and then transform the 'Fully Paid' values to 1 for the positive case, and the 'Charged Off' values to 0 for the negative case. 

We can use the method 'replace' to transform column values, and we can pass the replace method a nested mapping dictionary in the following format:

mapping_dict = {
    "date": {
        "january": 1,
        "february": 2,
        "march": 3
    }
}

df = df.replace(mapping_dict)

We also need to consider the 'class imbalance' between the positive and negative cases. While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off. This class imbalance is a common problem in binary classification, and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set, and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes. We will tackle the class imbalance later on.

In [11]:
# select only loans that are 'Fully Paid' and 'Charged Off'
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") 
                        | (loans_2007['loan_status'] == "Charged Off")]

In [12]:
# create the mapping dictionary
status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

# replace the values with the mapping dictionary
loans_2007 = loans_2007.replace(status_replace)

In [13]:
# check the 'loan_status' column which is now 0s and 1s
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,n,...,f,Jun-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,n,...,f,Sep-13,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,n,...,f,Jun-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,n,...,f,Apr-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,n,...,f,Jan-16,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


We will look at any columns that contain only one unique value, and remove them. These columns won't be useful for the model, since they don't add any information to each loan application.

We will compute the number of unique values in each column, and drop the columns that contain only one unique value. First we should drop the null values, then compute the number of unique values.

In [14]:
orig_columns = loans_2007.columns
drop_columns = []

In [15]:
for col in orig_columns:
    # drop null values in each column, and return the unique values
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)

# drop columns that have only 1 unique value
loans_2007 = loans_2007.drop(drop_columns, axis=1)

## Preparing The Features

We have removed all columns that contained redundant information, weren't useful for modeling, required too much processing, or leaked information from the future. 

We will now prepare the data for machine learning by focusing on handling missing values, converting categorical columns to numeric columns, and removing any other extraneous columns. This is because the mathematics underlying most machine learning models assumes the data is numerical, and contains no missing values.

We will compute the number of missing values. We can return the number of missing values by:

- using the method 'isnull' to return a Dataframe containing Boolean values (True if value is null, False is value is not null)
- then using the method 'sum 'to calculate the number of null values in each column.

In [16]:
loans = loans_2007

In [17]:
# show the number of null values for each column
null_counts = loans.isnull().sum()
print(null_counts)

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


Two columns have 50 or less rows with missing values, and 1 column, ''pub_rec_bankruptcies', contains 697 rows with missing values.

We will remove columns where more than 1% of the rows for that column contain a null value, and remove rows containing null values. So we will keep the following columns, and just remove rows containing missing values for them:

- title
- revol_util
- last_credit_pull_d

We will drop the 'pub_rec_bankruptcies' column entirely.

We will use the strategy of removing the 'pub_rec_bankruptcies' column first, then removing all rows containing any missing values. This way, we only remove the rows containing missing values for the 'title' and 'revol_util' columns but not the 'pub_rec_bankruptcies' column.

In [18]:
# drop the 'pub_rec_bankruptcies' COLUMN (axis = 1)
loans = loans.drop("pub_rec_bankruptcies", axis=1)

# drop ROWS (axis = 0) with missing values
loans = loans.dropna(axis=0)

# return the counts showing the numbers of each column per data type
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


The object columns that contain text need to be converted to numerical data.  We will select just the object columns, then display a sample row to get a better sense of how the values in each column are formatted.

In [19]:
# filter by the object/text columns
object_columns_df = loans.select_dtypes(include=["object"])

# show the first row
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line            Jan-85
revol_util                  83.70%
last_credit_pull_d          Jun-16
Name: 0, dtype: object


Some of the columns represent categorical values, but we should confirm by checking the number of unique values in those columns:

- home_ownership: home ownership status, can only be 1 of 4 categorical values
- verification_status: indicates if income was verified by Lending Club
- emp_length: number of years the borrower has been employed
- term: number of payments on the loan, either 36 or 60
- addr_state: borrower's state of residence
- purpose: a category provided by the borrower for the loan request
- title: loan title provided the borrower

There are some columns that represent numeric values, that need to be converted:

- int_rate: interest rate of the loan in %
- revol_util: revolving line utilisation rate, or the amount of credit the borrower is using relative to all available credit

Based on the first row's values for 'purpose' and 'title', it seems like these columns could reflect the same information. Let's explore the unique value counts separately to confirm this.

Lastly, some of the columns contain date values that would require feature engineering to be potentially useful:

- earliest_cr_line: the month the borrower's earliest credit line was opened
- last_credit_pull_d: the most recent month Lending Club pulled credit for this loan

Since these date features require feature engineering for modeling purposes, we will remove these date columns.

In [20]:
# look at unique value counts of the columnns containing categorical values
cols = ['home_ownership', 'verification_status', 
        'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans[c].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     311
OK     285
UT     249
KS     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
AK      76
WY      76
SD      60
VT  

The 'home_ownership', 'verification_status', 'emp_length', 'term', and 'addr_state' columns all contain multiple discrete values. 

We should clean the 'emp_length' column and treat it as numerical, since the values have ordering (i.e. 2 years of employment is less than 8 years).

We will look at the unique value counts for the purpose and title columns to understand which column we want to keep.

In [21]:
print(loans["purpose"].value_counts())
print(loans["title"].value_counts())

debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int64
Debt Consolidation                                                       2068
Debt Consolidation Loan                                                  1599
Personal Loan                                                             624
Consolidation                                                             488
debt consolidation                                                        466
Credit Card Consolidation                                                 345
Home Improvement                                                          336
Debt consolidation                

The 'home_ownership', 'verification_status', 'emp_length', and 'term' columns each contain some discrete categorical values. We should encode these columns as dummy variables and keep them.

The 'purpose' and 'title' columns do contain overlapping information, but we'll keep the 'purpose' column since it contains a few discrete values. In addition, the 'title' column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

We will use the a mapping to clean the emp_length column.

Lastly, the 'addr_state' column contains many discrete values, and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger, and could slow down how quickly the code runs. So we will remove this column.

In [22]:
# mapping to clean the 'emp_length' column
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

In [23]:
# remove the following columns
loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", 
                    "addr_state", "title"], axis=1)

In [24]:
# convert the 'int_rate' and 'revol_util' columns to float columns
    # strip the right trailing percent sign (%) using 'str.rstrip'
    # then use 'astype' to convert to float

loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")

In [25]:
# use the 'replace' method to clean the 'emp_length' column using dictionary above
loans = loans.replace(mapping_dict)

In [26]:
# see the cleansed 'emp_length' field
loans["emp_length"].head()

0    10
1     0
2    10
3    10
5     3
Name: emp_length, dtype: int64

We will now encode the 'home_ownership', 'verification_status', 'purpose', and 'term' columns as dummy variables, to use them in our model: 

- firstly, use the Pandas 'get_dummies' method to return a new Dataframe containing a new column for each dummy variable.
- secondly, use the 'concat' method to add these dummy columns back to the original Dataframe.
- thirdly, drop the original columns entirely using the drop method

In [27]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]

# return new dataframe containing new columns for each dummy variable
dummy_df = pd.get_dummies(loans[cat_columns])

# add these dummy columns back to original dataframe
loans = pd.concat([loans, dummy_df], axis=1)

# drop original columns
loans = loans.drop(cat_columns, axis=1)

Now we will experiment with training models, and evaluate accuracy using cross-validation.

## Making Predictions

We have cleansed and prepared a dataset that contains data on loans made to members of Lending Club. We want to generate features from the data, which can feed into a machine learning algorithm. The algorithm will make predictions about whether or not a loan will be paid off on time, which is contained in the 'loan_status' column.

So far, we have prepared the data, removed columns that had data leakage issues, contained redundant information, or required additional processing. We also cleaned features that had formatting issues, and converted categorical columns to dummy variables.

We noticed a class imbalance in our target column, 'loan_status'. There are about 6 times as many loans that were paid off on time (positive case, label of 1) than those that weren't (negative case, label of 0). Imbalances can cause issues with machine learning algorithms, where they appear to have high accuracy, but actually aren't learning from the training data. Because of its potential to cause issues, we need to keep the class imbalance in mind as we build machine learning models.

We will view a summary of the work we've done (using a pre-cleansed dataset).

In [28]:
import pandas as pd
loans = pd.read_csv(r"C:\projectdatasets\cleaned_loans_2007.csv")
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38708 entries, 0 to 38707
Data columns (total 38 columns):
loan_amnt                              38708 non-null int64
int_rate                               38708 non-null float64
installment                            38708 non-null float64
emp_length                             38708 non-null int64
annual_inc                             38708 non-null float64
loan_status                            38708 non-null int64
dti                                    38708 non-null float64
delinq_2yrs                            38708 non-null int64
inq_last_6mths                         38708 non-null int64
open_acc                               38708 non-null int64
pub_rec                                38708 non-null int64
revol_bal                              38708 non-null int64
revol_util                             38708 non-null float64
total_acc                              38708 non-null int64
home_ownership_MORTGAGE                38

Before we dive into predicting 'loan_status', we will explore the original question:
- can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

This is a binary classification problem, and converted the 'loan_status' column to 0s and 1s as a result. Before selecting an algorithm, we should select an error metric.

The error metric will determine when our model is performing well, and when it's performing poorly. We want to predict whether we should fund a loan. Our objective is to make money - we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. An error metric will help us determine if our algorithm will make us money, or lose us money.

In this case, we're concerned with 'false positives' and 'false negatives'. Both are misclassifications. With a false positive, we predict a loan will be paid off on time, but it actually isn't. This costs us money. With a false negative, we predict a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a profitable loan.

A conservative investor would want to minimize risk, and avoid false positives. They'd be more okay with missing out on opportunities (false negatives) than funding a risky loan (false positives).

We will calculate false positives and true positives in Python.

NOTE - THE BELOW IS SAMPLE CODE - DON'T RUN - NO 'PREDICTIONS' EXIST YET

In [ ]:
# number of false positives
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# number of true positives
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# number of false negatives
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# number of true negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

There is a class imbalance in the 'loan_status' column. There are 6 times as many loans that were paid off on time (1), than loans that weren't paid off on time (0). This causes an issue when using accuracy as a metric. This is because due to the class imbalance, a classifier may predict 1 for every row, and despite a few being wrong (and losing us money), the model is still has high accuracy.

So be aware of imbalanced classes in machine learning models, and adjust error metric accordingly. Here, we don't want to use accuracy, and should use metrics that tell us the number of false positives and false negatives. This means that we should optimize for:

- high recall (true positive rate)
- low fall-out (false positive rate)

We can calculate true positive rate, and false positive rate, using the numbers of true positives, true negatives, false negatives, and false positives.

- False Positive Rate (FPR) is the number of false positives divided by the number of false positives, plus the number of true negatives: fpr = fp / (fp + tn)

    This means "what percentage of my 1 predictions are incorrect?", or "what percentage of the loans that I fund would not be repaid?"
    

- True Positive Rate (TPR) is the number of true positives divided by the number of true positives, plus the number of false negatives: tpr = tp / (tp + fn)

    This means "what percentage of all the possible 1 predictions am I making?" or "what percentage of loans that could be funded would I fund?

NOTE - THE BELOW IS SAMPLE CODE - DON'T RUN - NO 'PREDICTIONS' EXIST YET

In [ ]:
import pandas as pd
import numpy

# predict all loans will be paid off on time
predictions = pd.Series(numpy.ones(loans.shape[0]))

# false positives
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# true positives
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# false negatives
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# true negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)    # this equals 1
print(fpr)    # this equals 1

Both fpr and tpr were 1. This is because we predicted 1 for each row. This means that we correctly identified all the good loans (true positive rate), but we also incorrectly identified all the bad loans (false positive rate). Now that we've setup error metrics, we can make predictions using a machine learning algorithm.

## Logistic Regression

Our cleaned dataset contains 41 columns, and are either the int64 or the float64 data type. There aren't any null values. So we can now apply any machine learning algorithm to our dataset.

Although we can build our own implementations of algorithms, it's easier and faster to use algorithms someone else has already written and tuned for high performance (so we will use the Scikit-learn library)

A good first algorithm to apply to binary classification problems is logistic regression, because:

- it's quick to train and we can iterate more quickly,
- it's less prone to overfitting than more complex models like decision trees,
- it's easy to interpret.

In [29]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

# create a list of columns
cols = loans.columns

# remove the predictor column 'loan_status'
train_cols = cols.drop("loan_status")

# create new dataframe of just the training columns (minus 'loan_status')
features = loans[train_cols]

# create new series of just the target column ('loan_status')
target = loans["loan_status"]

# fit a logistic regression and make predictions
lr.fit(features, target)
predictions = lr.predict(features)

C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


While we generated predictions, those predictions were overfit, because we generated predictions using the same data we trained our model on. When we use this to evaluate error, we get an unrealistically high depiction of how accurate the algorithm is, because it already "knows" the correct answers.

To get a realistic depiction, we will perform k-fold cross validation. Once we have cross validated predictions, we can compute the true positive rate, and false positive rate.

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

lr = LogisticRegression()

# ensure 3-fold cross validation is performed
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [31]:
# number of false positives
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# number of true positives
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# number of false negatives
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# number of true negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print(tpr)
print(fpr)

0.9989121566494424
0.9967943009795192


The 'fpr' and 'tpr' are what we'd expect, if the model was predicting all ones.

Unfortunately, even through we're not using accuracy as an error metric, the classifier is, and isn't accounting for the imbalance in the classes. There are a few ways to get a classifier to correct for imbalanced classes:

- use oversampling and undersampling, to ensure the classifier gets input that has a balanced number of each class
- tell the classifier to penalise misclassifications of the less prevalent class more than the other class

We will consider oversampling and undersampling first. They involve taking a sample that contains equal numbers of rows where 'loan_status' is 0, and where 'loan_status' is 1. This way, the classifier is forced to make actual predictions, since predicting all 1s or all 0s will only result in 50% accuracy at most. The downside of this, is that since it has to preserve an equal ratio, you have to either:

- throw out many rows of data i.e. where 'loan_status' is 1.
- copy rows i.e. copy rows where 'loan_status' is 0 to equalise the rows
- generate fake data i.e. generate new rows where 'loan_status' is 0.

None of these techniques are easy. 

The second method is telling the classifier to penalise certain rows more, which is easier to implement. We can do this by setting the 'class_weight' parameter to 'balanced' when creating the LogisticRegression instance. This tells it to penalise the misclassification of the minority class during training. So the logistic regression classifier pays more attention to correctly classifying rows where 'loan_status' is 0. This lowers accuracy when 'loan_status' is 1, but raises accuracy when 'loan_status' is 0.

The penalty is set to be inversely proportional to the class frequencies. Correctly classifying a row where 'loan_status' is 0 is 6 times more important than correctly classifying a row where 'loan_status' is 1.

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

# this time, set 'class_weight' to 'balanced'
lr = LogisticRegression(class_weight="balanced")

predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [33]:
# number of false positives
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# number of true positives
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# number of false negatives
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# number of true negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print(tpr)
print(fpr)

0.6644909799655516
0.38913624220837045


We improved the false positive rate by balancing the classes, which reduced the true positive rate. Our true positive rate is now around 66%, and our false positive rate is 38%. From a conservative investor's standpoint, it's reassuring that the false positive rate is lower, because it means that we'll be able to do a better job at avoiding bad loans, than if we funded everything. However, we'd only ever decide to fund 63% of total loans (true positive rate), so we'd immediately reject a good amount of loans.

We can lower the false positive rate further by assigning a harsher penalty for misclassifying the negative class. While setting 'class_weight' to 'balanced' will automatically set a penalty based on the number of 1s and 0s in the column, we can also set a manual penalty. The penalty scikit-learn imposed for misclassifying a 0 would have been around 5.89 (since there are 5.89 times as many 1s as 0s).

We can also specify a penalty manually if we want to adjust the rates more. To do this, pass in a dictionary of penalty values to the 'class_weight' parameter.

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

# dictionary will impose penalty of 10 for misclassifying a 0, and 1 for misclassifying a 1
penalty = {
    0: 10,
    1: 1
}

# manually pass in the 'penalty' dictionary to 'class_weight'
lr = LogisticRegression(class_weight=penalty)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [35]:
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# number of true positives
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# number of false negatives
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# number of true negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print(tpr)
print(fpr)

0.2356691747499471
0.08708815672306322


Assigning manual penalties lowered the false positive rate to 8%, and thus lowered our risk. This comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

While we could tweak the penalties further, it's best to try a different (more complex) model, like random forest. Random forests can work with nonlinear data, and learn complex conditionals. Logistic regressions are only able to work with linear data. Training a random forest algorithm may enable us to get more accuracy due to columns that correlate nonlinearly with 'loan_status'.

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

rf = RandomForestClassifier(class_weight="balanced", random_state=1)
predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Chris\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [37]:
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# number of true positives
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# number of false negatives
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# number of true negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print(tpr)
print(fpr)

0.9709304082434351
0.9271593944790739


Unfortunately, a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.

Ultimately, our best model had a false positive rate of 8%, and a true positive rate of 23%. For a conservative investor, they will make money as long as the interest rate is high enough to offset the losses from 8% of borrowers defaulting, and that the pool of 23% of borrowers is large enough to make enough interest money to offset the losses.

If we had randomly picked loans to fund, borrowers would have defaulted on 14.5% of them, and our model is better than that, although we're excluding more loans than a random strategy would. We can still improve by:

- tweaking the penalties further
- try models other than a random forest and logistic regression
- use some of the columns we discarded to generate better features
- ensemble multiple models to get more accurate predictions
- tune the parameters of the algorithm to achieve higher performance